In [ ]:
from pathlib import Path
import rioxarray as rx
import xarray as xr
import geopandas as gpd
from geocube.api.core import make_geocube
import os
os.chdir("/home/rave/temporal_mixing")

def list_downloaded_sr_paths(data_path):
    scene_paths = list(data_path.glob("*udm2"))
    sr_paths = []
    for path in scene_paths:
        sr_path = next(path.glob("files/*SR_clip.tif"))
        sr_paths.append(sr_path)
    return sr_paths

def reproject_path_lists(scene_paths, aoi_xarr):
    """aoi_xarr is converted gdf to rioxarray Datarray, reprojects all scenes onto same grid
    """
    matched_scene_xarrs = []
    for path in scene_paths:
        scene_xarr = rx.open_rasterio(path, masked=True)
        scene_xarr_matched = scene_xarr.rio.reproject_match(aoi_xarr)
        matched_scene_xarrs.append(scene_xarr_matched)
    return matched_scene_xarrs

data_path = Path("data/downloads")
sr_paths = list_downloaded_sr_paths(data_path)

san_pedro_aoi = gpd.read_file("data/san_pedro_smaller_section.geojson")
san_pedro_aoi["id"] = 1

#gdf to xarr for reprojecting and grid coordinate matching
aoi_xarr = make_geocube(san_pedro_aoi, resolution=(3.0, -3.0), output_crs= "EPSG:32612", measurements= ["id"])
aoi_xarr = aoi_xarr["id"]

In [ ]:
scene_xarrs_list = reproject_path_lists(sr_path, aoi_xarr)

In [ ]:
# should scenes on same day be merged together? prbly. first need to determine if there are duplicates
stacked_planet_scenes = xr.concat(scene_xarrs_list, dim="scene")

In [ ]:
stacked_planet_scenes.rio.to_raster("outputs/stacked_planet_2018_06_2019_06.tif")

In [ ]:
test_0 = rx.open_rasterio(sr_paths[0]).sel(band=slice(1,3))
test_0.plot.imshow(robust=True)

In [ ]:
test_100 = rx.open_rasterio(sr_paths[100])
test_100.sel(band=slice(1,3)).plot.imshow(robust=True)

In [ ]:
test_100.rio.reproject_match(test_0).plot.imshow(robust=True)

In [ ]:
test_100_match = test_100.rio.reproject_match(aoi_xarr)

In [ ]:
test_100_match.plot.imshow(robust=True)